In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [2]:

# DATA MANIPULATION
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import math

# MACHINE LEARNING - SCIKIT-LEARN
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE, RFECV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline

# ADVANCED ML LIBRARIES
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier

# VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# CONFIGURATION FOR PLOTS
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# TEXT PROCESSING (NLP)
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud

# DEEP LEARNING 
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv2D, MaxPooling2D, Flatten
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    print("TensorFlow disponible")
except ImportError:
    print("TensorFlow no disponible en este entorno")


# UTILITIES
import os
import sys
import warnings
import itertools
from datetime import datetime, timedelta
import time
from collections import Counter
import pickle
import joblib

# JUPYTER SPECIFIC
from IPython.display import display, HTML
from tqdm.notebook import tqdm

# SUPPRESS WARNINGS
warnings.filterwarnings('ignore')

# PANDAS CONFIGURATION
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# NUMPY CONFIGURATION
np.random.seed(42)

# PLOTLY CONFIGURATION
pyo.init_notebook_mode(connected=True)

print("✅ Todas las librerías importadas correctamente!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"🤖 Scikit-learn version: {__import__('sklearn').__version__}")
print(f"📈 Matplotlib version: {__import__('matplotlib').__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")


def quick_info(df):
    """Información rápida del dataset"""
    print(f"📊 Dataset Shape: {df.shape}")
    print(f"🔢 Columnas numéricas: {df.select_dtypes(include=[np.number]).columns.tolist()}")
    print(f"📝 Columnas categóricas: {df.select_dtypes(include=['object']).columns.tolist()}")
    print(f"❌ Valores nulos por columna:")
    print(df.isnull().sum()[df.isnull().sum() > 0])



2025-08-11 22:31:32.844581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754951493.112885      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754951493.186260      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow disponible


✅ Todas las librerías importadas correctamente!
📊 Pandas version: 2.2.3
🔢 NumPy version: 1.26.4
🤖 Scikit-learn version: 1.2.2
📈 Matplotlib version: 3.7.2
🎨 Seaborn version: 0.12.2


In [3]:
#https://www.kaggle.com/competitions/playground-series-s5e8/leaderboard? concurso
train = pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
test =  pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")


https://www.kaggle.com/competitions/playground-series-s5e8/overview

*luego de cargar los datos , verificar si hay datos faltantes y tratar de hacer un randomforest lo mas rapido posible
para ver como viene el modelo y el leaderboard , despues profundizare para entender el dataset y ver como hacer
un feature eng y proceder a futuro


In [4]:
#age      : Age of the client (numeric)
#job      : Type of job (categorical: "admin.", "blue-collar", "entrepreneur", etc.)
#marital  : Marital status (categorical: "married", "single", "divorced")
#education: Level of education (categorical: "primary", "secondary", "tertiary", "unknown")
#default  : Has credit in default? (categorical: "yes", "no")
#balance  : Average yearly balance in euros (numeric)
#housing  : Has a housing loan? (categorical: "yes", "no")
#loan     : Has a personal loan? (categorical: "yes", "no")
#contact  : Type of communication contact (categorical: "unknown", "telephone", "cellular")
#day      : Last contact day of the month (numeric, 1-31)
#month    : Last contact month of the year (categorical: "jan", "feb", "mar", …, "dec")
#duration : Last contact duration in seconds (numeric)
#campaign : Number of contacts performed during this campaign (numeric)
#pdays    : Number of days since the client was last contacted from a previous campaign (numeric; -1 means the client was not previously contacted)
#previous : Number of contacts performed before this campaign (numeric)
#poutcome : Outcome of the previous marketing campaign (categorical: "unknown", "other", "failure", "success")
#y        : The target variable, whether the client subscribed to a term deposit (binary: "yes", "no")

In [5]:
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [6]:
test.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
1,750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
2,750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
3,750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
4,750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown


In [7]:
quick_info(train)

📊 Dataset Shape: (750000, 18)
🔢 Columnas numéricas: ['id', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'y']
📝 Columnas categóricas: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
❌ Valores nulos por columna:
Series([], dtype: int64)


In [8]:
quick_info(test)

📊 Dataset Shape: (250000, 17)
🔢 Columnas numéricas: ['id', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
📝 Columnas categóricas: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
❌ Valores nulos por columna:
Series([], dtype: int64)


In [9]:
#train["job"].value_counts()       # +12-1 management/blue-collar/technician/admin./services/retired/self-employed/entrepeneur/
                                  #       unemployed/housemaid/student/unknown
#train["marital"].value_counts()   # +3-1  married/single/divorced
#train["education"].value_counts() # +4-1 secondary/tertiary/primary/unknown
#train["default"].value_counts()   # binario reemplazo no = 0 (98.3%), yes  = 1 (1.7%)
#train["housing"].value_counts()   # binario reemplazo no = 0 (45.16%), yes  = 1 (54.83%)
#train["loan"].value_counts()      # binario reemplazo no = 0 (86.03%), yes  = 1 (13.97%)
#train["contact"].value_counts()   # one-hot 2 columnas , cellular/unknown/telephone si ambos son 0 entonces es unknown
#train["month"].value_counts()     # lo primero que pienso es ordinal pero hay mucha dif entre los valores
#                                  # la tasa de conversion de abajo me empuja a que sea one-hot o target encoding
#                                  # dado que mas tarde tmb quiero hacer una NN , hago one-hot y despues pruebo con Target
#month
#may    228411
#aug    128859
#jul    110647
#jun     93670
#nov     66062
#apr     41319
#feb     37611
#jan     18937
#oct      9204
#sep      7409
#mar      5802
#dec      2069
#train["poutcome"].value_counts() # one-hot 3 columnas unknown/failure/succes/other si todos son 0 entonces es unknown

In [10]:
# Calcular tasa de conversión por mes
conversion_by_month = train.groupby('month')['y'].agg(['count', 'sum', 'mean'])
conversion_by_month.columns = ['total_contacts', 'conversions', 'conversion_rate']
conversion_by_month = conversion_by_month.sort_values('conversion_rate', ascending=False)

print(conversion_by_month)

       total_contacts  conversions  conversion_rate
month                                              
mar              5802         3315         0.571355
sep              7409         3962         0.534755
dec              2069         1062         0.513291
oct              9204         4510         0.490004
apr             41319         9737         0.235654
feb             37611         7778         0.206801
jan             18937         2351         0.124148
aug            128859        14453         0.112161
nov             66062         7254         0.109806
jun             93670         9716         0.103726
jul            110647        10052         0.090847
may            228411        16298         0.071354


In [11]:
#feat Eng
train.describe(include="all")

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
count,750000.000000,750000.000000,750000,750000,750000,750000,750000.000000,750000,750000,750000,750000.000000,750000,750000.000000,750000.000000,750000.000000,750000.000000,750000,750000.000000
unique,NaN,NaN,12,3,4,2,NaN,2,2,3,NaN,12,NaN,NaN,NaN,NaN,4,NaN
top,NaN,NaN,management,married,secondary,no,NaN,yes,no,cellular,NaN,may,NaN,NaN,NaN,NaN,unknown,NaN
freq,NaN,NaN,175541,480759,401683,737151,NaN,411288,645023,486655,NaN,228411,NaN,NaN,NaN,NaN,672450,NaN
mean,374999.500000,40.926395,NaN,NaN,NaN,NaN,1204.067397,NaN,NaN,NaN,16.117209,NaN,256.229144,2.577008,22.412733,0.298545,NaN,0.120651
std,216506.495284,10.098829,NaN,NaN,NaN,NaN,2836.096759,NaN,NaN,NaN,8.250832,NaN,272.555662,2.718514,77.319998,1.335926,NaN,0.325721
min,0.000000,18.000000,NaN,NaN,NaN,NaN,-8019.000000,NaN,NaN,NaN,1.000000,NaN,1.000000,1.000000,-1.000000,0.000000,NaN,0.000000
25%,187499.750000,33.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,9.000000,NaN,91.000000,1.000000,-1.000000,0.000000,NaN,0.000000
50%,374999.500000,39.000000,NaN,NaN,NaN,NaN,634.000000,NaN,NaN,NaN,17.000000,NaN,133.000000,2.000000,-1.000000,0.000000,NaN,0.000000
75%,562499.250000,48.000000,NaN,NaN,NaN,NaN,1390.000000,NaN,NaN,NaN,21.000000,NaN,361.000000,3.000000,-1.000000,0.000000,NaN,0.000000


In [12]:
#   # Feature Importance XGB
# 26   poutcome_success    0.295378
# 14          month_mar    0.095745
# 7            duration    0.074167
# 11   contact_cellular    0.067707
# 4             housing    0.044851
# 21          month_oct    0.037620
# 15          month_apr    0.034245
# 17          month_jun    0.033480
# 20          month_sep    0.030168
# 12  contact_telephone    0.024734
# 5                loan    0.022850
# 27    job_blue-collar    0.020831
# 16          month_may    0.019547
# 13          month_feb    0.017789
# 18          month_jul    0.014844
# 22          month_nov    0.013827
# 23          month_dec    0.013768
# 19          month_aug    0.012835
# 9               pdays    0.009650
# 10           previous    0.009134


In [13]:


# # 1. Histograma general de duration
# plt.figure(figsize=(12, 4))

# plt.subplot(1, 3, 1)
# plt.hist(train['duration'], bins=100, alpha=0.7, edgecolor='black')
# plt.title('Duration - Todos los casos')
# plt.xlabel('Duration')
# plt.ylabel('Frecuencia')

# # 2. Histograma para y >= 0.5 (casos positivos)
# plt.subplot(1, 3, 2)
# duration_positive = train[train['y'] >= 0.5]['duration']
# plt.hist(duration_positive, bins=100, alpha=0.7, color='green', edgecolor='black')
# plt.title('Duration - y >= 0.5')
# plt.xlabel('Duration')
# plt.ylabel('Frecuencia')

# # 3. Histograma para y < 0.5 (casos negativos)
# plt.subplot(1, 3, 3)
# duration_negative = train[train['y'] < 0.5]['duration']
# plt.hist(duration_negative, bins=range(0,1000,50), alpha=0.7, color='red', edgecolor='black')
# plt.title('Duration - y < 0.5')
# plt.xlabel('Duration')
# plt.ylabel('Frecuencia')

# plt.tight_layout()
# plt.show()

# # Stats rápidas
# print(f"Duration media (y>=0.5): {duration_positive.mean():.1f}")
# print(f"Duration media (y<0.5): {duration_negative.mean():.1f}")
# print(f"Duration mediana (y>=0.5): {duration_positive.median():.1f}")
# print(f"Duration mediana (y<0.5): {duration_negative.median():.1f}")

In [14]:

# # HISTOGRAMAS
# plt.figure(figsize=(15, 8))

# # 1. Histograma general de balance
# plt.subplot(2, 3, 1)
# plt.hist(train['balance'], bins=50, alpha=0.7, edgecolor='black')
# plt.title('Balance - Todos los casos')
# plt.xlabel('Balance')
# plt.ylabel('Frecuencia')

# # 2. Histograma para y >= 0.5 (casos positivos)
# plt.subplot(2, 3, 2)
# balance_positive = train[train['y'] >= 0.5]['balance']
# plt.hist(balance_positive, bins=50, alpha=0.7, color='green', edgecolor='black')
# plt.title('Balance - y >= 0.5')
# plt.xlabel('Balance')
# plt.ylabel('Frecuencia')

# # 3. Histograma para y < 0.5 (casos negativos)
# plt.subplot(2, 3, 3)
# balance_negative = train[train['y'] < 0.5]['balance']
# plt.hist(balance_negative, bins=range(0,20000,1000), alpha=0.7, color='red', edgecolor='black')
# plt.title('Balance - y < 0.5')
# plt.xlabel('Balance')
# plt.ylabel('Frecuencia')

# # BOXPLOTS
# plt.subplot(2, 3, 4)
# plt.boxplot(train['balance'])
# plt.title('Balance - Todos')
# plt.ylabel('Balance')

# plt.subplot(2, 3, 5)
# plt.boxplot(balance_positive, patch_artist=True, 
#            boxprops=dict(facecolor='lightgreen'))
# plt.title('Balance - y >= 0.5')
# plt.ylabel('Balance')

# plt.subplot(2, 3, 6)
# plt.boxplot(balance_negative, patch_artist=True,
#            boxprops=dict(facecolor='lightcoral'))
# plt.title('Balance - y < 0.5')
# plt.ylabel('Balance')

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS DESCRIPTIVAS
# def print_quartiles(data, label):
#     q25 = np.percentile(data, 25)
#     q50 = np.percentile(data, 50)  # mediana
#     q75 = np.percentile(data, 75)
#     print(f"\n{label}:")
#     print(f"  Q1 (25%): {q25:.1f}")
#     print(f"  Q2 (50% - Mediana): {q50:.1f}")
#     print(f"  Q3 (75%): {q75:.1f}")
#     print(f"  Media: {data.mean():.1f}")
#     print(f"  Min: {data.min():.1f}")
#     print(f"  Max: {data.max():.1f}")
#     print(f"  Count: {len(data)}")

# # Imprimir estadísticas
# print_quartiles(train['balance'], "BALANCE - TODOS")
# print_quartiles(balance_positive, "BALANCE - y >= 0.5")
# print_quartiles(balance_negative, "BALANCE - y < 0.5")

In [15]:

# plt.figure(figsize=(15, 8))

# # 1. Distribución general de contact
# plt.subplot(2, 3, 1)
# contact_counts = train['contact'].value_counts()
# plt.bar(contact_counts.index, contact_counts.values, alpha=0.7, edgecolor='black')
# plt.title('Contact - Todos los casos')
# plt.xlabel('Contact Type')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# contact_positive = train[train['y'] >= 0.5]['contact'].value_counts()
# plt.bar(contact_positive.index, contact_positive.values, alpha=0.7, color='green', edgecolor='black')
# plt.title('Contact - y >= 0.5')
# plt.xlabel('Contact Type')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# contact_negative = train[train['y'] < 0.5]['contact'].value_counts()
# plt.bar(contact_negative.index, contact_negative.values, alpha=0.7, color='red', edgecolor='black')
# plt.title('Contact - y < 0.5')
# plt.xlabel('Contact Type')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 4. Tasas de conversión por contact type
# plt.subplot(2, 3, 4)
# conversion_rates = train.groupby('contact')['y'].agg(['mean', 'count'])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Contact')
# plt.xlabel('Contact Type')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Distribución porcentual
# plt.subplot(2, 3, 5)
# contact_pct = train['contact'].value_counts(normalize=True) * 100
# plt.pie(contact_pct.values, labels=contact_pct.index, autopct='%1.1f%%')
# plt.title('Contact - Distribución %')

# # 6. Comparación lado a lado
# plt.subplot(2, 3, 6)
# comparison_data = pd.DataFrame({
#     'y >= 0.5': train[train['y'] >= 0.5]['contact'].value_counts(),
#     'y < 0.5': train[train['y'] < 0.5]['contact'].value_counts()
# }).fillna(0)

# comparison_data.plot(kind='bar', ax=plt.gca(), color=['green', 'red'], alpha=0.7)
# plt.title('Contact - Comparación')
# plt.xlabel('Contact Type')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE CONTACT ===")
# print("\nTodos los casos:")
# print(train['contact'].value_counts())
# print(f"\nPorcentajes:")
# print((train['contact'].value_counts(normalize=True) * 100).round(1))

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['contact'].value_counts())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['contact'].value_counts())

# print(f"\nTasas de conversión por contact:")
# conversion_stats = train.groupby('contact')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}))

In [16]:
# plt.figure(figsize=(15, 8))

# # 1. Distribución general de age
# plt.subplot(2, 3, 1)
# plt.hist(train['age'], bins=30, alpha=0.7, edgecolor='black')
# plt.title('Age - Todos los casos')
# plt.xlabel('Age')
# plt.ylabel('Frecuencia')

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# plt.hist(train[train['y'] >= 0.5]['age'], bins=30, alpha=0.7, color='green', edgecolor='black')
# plt.title('Age - y >= 0.5')
# plt.xlabel('Age')
# plt.ylabel('Frecuencia')

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# plt.hist(train[train['y'] < 0.5]['age'], bins=30, alpha=0.7, color='red', edgecolor='black')
# plt.title('Age - y < 0.5')
# plt.xlabel('Age')
# plt.ylabel('Frecuencia')

# # 4. Tasas de conversión por age (binned)
# plt.subplot(2, 3, 4)
# age_bins = pd.cut(train['age'], bins=20)
# conversion_rates = train.groupby(age_bins)['y'].agg(['mean', 'count'])
# bin_centers = [interval.mid for interval in conversion_rates.index]
# plt.bar(bin_centers, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black', width=2)
# plt.title('Tasa de Conversión por Age')
# plt.xlabel('Age')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Boxplot comparativo
# plt.subplot(2, 3, 5)
# data_to_plot = [train[train['y'] >= 0.5]['age'], train[train['y'] < 0.5]['age']]
# box_plot = plt.boxplot(data_to_plot, patch_artist=True, labels=['y >= 0.5', 'y < 0.5'])
# box_plot['boxes'][0].set_facecolor('green')
# box_plot['boxes'][1].set_facecolor('red')
# plt.title('Age - Distribución por Clase')
# plt.ylabel('Age')

# # 6. Comparación de densidades
# plt.subplot(2, 3, 6)
# train[train['y'] >= 0.5]['age'].plot(kind='density', alpha=0.7, color='green', label='y >= 0.5')
# train[train['y'] < 0.5]['age'].plot(kind='density', alpha=0.7, color='red', label='y < 0.5')
# plt.title('Age - Densidades')
# plt.xlabel('Age')
# plt.ylabel('Densidad')
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE AGE ===")
# print("\nTodos los casos:")
# print(train['age'].describe())

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['age'].describe())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['age'].describe())

# print(f"\nTasas de conversión por rangos de age:")
# age_ranges = pd.cut(train['age'], bins=[0, 25, 35, 45, 55, 65, 100], labels=['<25', '25-34', '35-44', '45-54', '55-64', '65+'])
# conversion_stats = train.groupby(age_ranges)['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}))

# print(f"\nCorrelación con target:")
# print(f"Correlación Pearson: {train['age'].corr(train['y']):.4f}")

# # Análisis de percentiles para identificar puntos de corte óptimos
# print(f"\nPercentiles de age:")
# percentiles = [10, 25, 50, 75, 90, 95, 99]
# for p in percentiles:
#     print(f"P{p}: {train['age'].quantile(p/100):.1f}")

In [17]:
# plt.figure(figsize=(15, 8))

# # 1. Distribución general de job
# plt.subplot(2, 3, 1)
# job_counts = train['job'].value_counts()
# plt.bar(job_counts.index, job_counts.values, alpha=0.7, edgecolor='black')
# plt.title('Job - Todos los casos')
# plt.xlabel('Job Type')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# job_positive = train[train['y'] >= 0.5]['job'].value_counts()
# plt.bar(job_positive.index, job_positive.values, alpha=0.7, color='green', edgecolor='black')
# plt.title('Job - y >= 0.5')
# plt.xlabel('Job Type')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# job_negative = train[train['y'] < 0.5]['job'].value_counts()
# plt.bar(job_negative.index, job_negative.values, alpha=0.7, color='red', edgecolor='black')
# plt.title('Job - y < 0.5')
# plt.xlabel('Job Type')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 4. Tasas de conversión por job type
# plt.subplot(2, 3, 4)
# conversion_rates = train.groupby('job')['y'].agg(['mean', 'count'])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Job')
# plt.xlabel('Job Type')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Distribución porcentual
# plt.subplot(2, 3, 5)
# job_pct = train['job'].value_counts(normalize=True) * 100
# plt.pie(job_pct.values, labels=job_pct.index, autopct='%1.1f%%')
# plt.title('Job - Distribución %')

# # 6. Comparación lado a lado
# plt.subplot(2, 3, 6)
# comparison_data = pd.DataFrame({
#     'y >= 0.5': train[train['y'] >= 0.5]['job'].value_counts(),
#     'y < 0.5': train[train['y'] < 0.5]['job'].value_counts()
# }).fillna(0)
# comparison_data.plot(kind='bar', ax=plt.gca(), color=['green', 'red'], alpha=0.7)
# plt.title('Job - Comparación')
# plt.xlabel('Job Type')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE JOB ===")
# print("\nTodos los casos:")
# print(train['job'].value_counts())

# print(f"\nPorcentajes:")
# print((train['job'].value_counts(normalize=True) * 100).round(1))

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['job'].value_counts())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['job'].value_counts())

# print(f"\nTasas de conversión por job:")
# conversion_stats = train.groupby('job')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}).sort_values('conversion_rate_%', ascending=False))

In [18]:
# plt.figure(figsize=(15, 8))

# # 1. Distribución general de marital
# plt.subplot(2, 3, 1)
# marital_counts = train['marital'].value_counts()
# plt.bar(marital_counts.index, marital_counts.values, alpha=0.7, edgecolor='black')
# plt.title('Marital - Todos los casos')
# plt.xlabel('Marital Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# marital_positive = train[train['y'] >= 0.5]['marital'].value_counts()
# plt.bar(marital_positive.index, marital_positive.values, alpha=0.7, color='green', edgecolor='black')
# plt.title('Marital - y >= 0.5')
# plt.xlabel('Marital Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# marital_negative = train[train['y'] < 0.5]['marital'].value_counts()
# plt.bar(marital_negative.index, marital_negative.values, alpha=0.7, color='red', edgecolor='black')
# plt.title('Marital - y < 0.5')
# plt.xlabel('Marital Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 4. Tasas de conversión por marital status
# plt.subplot(2, 3, 4)
# conversion_rates = train.groupby('marital')['y'].agg(['mean', 'count'])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Marital')
# plt.xlabel('Marital Status')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Distribución porcentual
# plt.subplot(2, 3, 5)
# marital_pct = train['marital'].value_counts(normalize=True) * 100
# plt.pie(marital_pct.values, labels=marital_pct.index, autopct='%1.1f%%')
# plt.title('Marital - Distribución %')

# # 6. Comparación lado a lado
# plt.subplot(2, 3, 6)
# comparison_data = pd.DataFrame({
#     'y >= 0.5': train[train['y'] >= 0.5]['marital'].value_counts(),
#     'y < 0.5': train[train['y'] < 0.5]['marital'].value_counts()
# }).fillna(0)
# comparison_data.plot(kind='bar', ax=plt.gca(), color=['green', 'red'], alpha=0.7)
# plt.title('Marital - Comparación')
# plt.xlabel('Marital Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE MARITAL ===")
# print("\nTodos los casos:")
# print(train['marital'].value_counts())

# print(f"\nPorcentajes:")
# print((train['marital'].value_counts(normalize=True) * 100).round(1))

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['marital'].value_counts())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['marital'].value_counts())

# print(f"\nTasas de conversión por marital:")
# conversion_stats = train.groupby('marital')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}).sort_values('conversion_rate_%', ascending=False))

In [19]:
# plt.figure(figsize=(15, 8))

# # 1. Distribución general de education
# plt.subplot(2, 3, 1)
# education_counts = train['education'].value_counts()
# plt.bar(education_counts.index, education_counts.values, alpha=0.7, edgecolor='black')
# plt.title('Education - Todos los casos')
# plt.xlabel('Education Level')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# education_positive = train[train['y'] >= 0.5]['education'].value_counts()
# plt.bar(education_positive.index, education_positive.values, alpha=0.7, color='green', edgecolor='black')
# plt.title('Education - y >= 0.5')
# plt.xlabel('Education Level')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# education_negative = train[train['y'] < 0.5]['education'].value_counts()
# plt.bar(education_negative.index, education_negative.values, alpha=0.7, color='red', edgecolor='black')
# plt.title('Education - y < 0.5')
# plt.xlabel('Education Level')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 4. Tasas de conversión por education level
# plt.subplot(2, 3, 4)
# conversion_rates = train.groupby('education')['y'].agg(['mean', 'count'])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Education')
# plt.xlabel('Education Level')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Distribución porcentual
# plt.subplot(2, 3, 5)
# education_pct = train['education'].value_counts(normalize=True) * 100
# plt.pie(education_pct.values, labels=education_pct.index, autopct='%1.1f%%')
# plt.title('Education - Distribución %')

# # 6. Comparación lado a lado
# plt.subplot(2, 3, 6)
# comparison_data = pd.DataFrame({
#     'y >= 0.5': train[train['y'] >= 0.5]['education'].value_counts(),
#     'y < 0.5': train[train['y'] < 0.5]['education'].value_counts()
# }).fillna(0)
# comparison_data.plot(kind='bar', ax=plt.gca(), color=['green', 'red'], alpha=0.7)
# plt.title('Education - Comparación')
# plt.xlabel('Education Level')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE EDUCATION ===")
# print("\nTodos los casos:")
# print(train['education'].value_counts())

# print(f"\nPorcentajes:")
# print((train['education'].value_counts(normalize=True) * 100).round(1))

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['education'].value_counts())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['education'].value_counts())

# print(f"\nTasas de conversión por education:")
# conversion_stats = train.groupby('education')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}).sort_values('conversion_rate_%', ascending=False))

In [20]:
# plt.figure(figsize=(15, 8))

# # 1. Distribución general de default
# plt.subplot(2, 3, 1)
# default_counts = train['default'].value_counts()
# plt.bar(default_counts.index, default_counts.values, alpha=0.7, edgecolor='black')
# plt.title('Default - Todos los casos')
# plt.xlabel('Default Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# default_positive = train[train['y'] >= 0.5]['default'].value_counts()
# plt.bar(default_positive.index, default_positive.values, alpha=0.7, color='green', edgecolor='black')
# plt.title('Default - y >= 0.5')
# plt.xlabel('Default Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# default_negative = train[train['y'] < 0.5]['default'].value_counts()
# plt.bar(default_negative.index, default_negative.values, alpha=0.7, color='red', edgecolor='black')
# plt.title('Default - y < 0.5')
# plt.xlabel('Default Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 4. Tasas de conversión por default status
# plt.subplot(2, 3, 4)
# conversion_rates = train.groupby('default')['y'].agg(['mean', 'count'])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Default')
# plt.xlabel('Default Status')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Distribución porcentual
# plt.subplot(2, 3, 5)
# default_pct = train['default'].value_counts(normalize=True) * 100
# plt.pie(default_pct.values, labels=default_pct.index, autopct='%1.1f%%')
# plt.title('Default - Distribución %')

# # 6. Comparación lado a lado
# plt.subplot(2, 3, 6)
# comparison_data = pd.DataFrame({
#     'y >= 0.5': train[train['y'] >= 0.5]['default'].value_counts(),
#     'y < 0.5': train[train['y'] < 0.5]['default'].value_counts()
# }).fillna(0)
# comparison_data.plot(kind='bar', ax=plt.gca(), color=['green', 'red'], alpha=0.7)
# plt.title('Default - Comparación')
# plt.xlabel('Default Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE DEFAULT ===")
# print("\nTodos los casos:")
# print(train['default'].value_counts())

# print(f"\nPorcentajes:")
# print((train['default'].value_counts(normalize=True) * 100).round(1))

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['default'].value_counts())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['default'].value_counts())

# print(f"\nTasas de conversión por default:")
# conversion_stats = train.groupby('default')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}).sort_values('conversion_rate_%', ascending=False))

In [21]:
# plt.figure(figsize=(15, 8))

# # 1. Distribución general de default
# plt.subplot(2, 3, 1)
# default_counts = train['default'].value_counts()
# plt.bar(default_counts.index, default_counts.values, alpha=0.7, edgecolor='black')
# plt.title('Default - Todos los casos')
# plt.xlabel('Default Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# default_positive = train[train['y'] >= 0.5]['default'].value_counts()
# plt.bar(default_positive.index, default_positive.values, alpha=0.7, color='green', edgecolor='black')
# plt.title('Default - y >= 0.5')
# plt.xlabel('Default Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# default_negative = train[train['y'] < 0.5]['default'].value_counts()
# plt.bar(default_negative.index, default_negative.values, alpha=0.7, color='red', edgecolor='black')
# plt.title('Default - y < 0.5')
# plt.xlabel('Default Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 4. Tasas de conversión por default status
# plt.subplot(2, 3, 4)
# conversion_rates = train.groupby('default')['y'].agg(['mean', 'count'])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Default')
# plt.xlabel('Default Status')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Distribución porcentual
# plt.subplot(2, 3, 5)
# default_pct = train['default'].value_counts(normalize=True) * 100
# plt.pie(default_pct.values, labels=default_pct.index, autopct='%1.1f%%')
# plt.title('Default - Distribución %')

# # 6. Comparación lado a lado
# plt.subplot(2, 3, 6)
# comparison_data = pd.DataFrame({
#     'y >= 0.5': train[train['y'] >= 0.5]['default'].value_counts(),
#     'y < 0.5': train[train['y'] < 0.5]['default'].value_counts()
# }).fillna(0)
# comparison_data.plot(kind='bar', ax=plt.gca(), color=['green', 'red'], alpha=0.7)
# plt.title('Default - Comparación')
# plt.xlabel('Default Status')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE HOUSING ===")
# print("\nTodos los casos:")
# print(train['housing'].value_counts())

# print(f"\nPorcentajes:")
# print((train['housing'].value_counts(normalize=True) * 100).round(1))

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['housing'].value_counts())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['housing'].value_counts())

# print(f"\nTasas de conversión por housing:")
# conversion_stats = train.groupby('housing')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}).sort_values('conversion_rate_%', ascending=False))

In [22]:
# plt.figure(figsize=(15, 8))

# # 1. Distribución general de loan
# plt.subplot(2, 3, 1)
# loan_counts = train['loan'].value_counts()
# plt.bar(loan_counts.index, loan_counts.values, alpha=0.7, edgecolor='black')
# plt.title('Loan - Todos los casos')
# plt.xlabel('Personal Loan')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# loan_positive = train[train['y'] >= 0.5]['loan'].value_counts()
# plt.bar(loan_positive.index, loan_positive.values, alpha=0.7, color='green', edgecolor='black')
# plt.title('Loan - y >= 0.5')
# plt.xlabel('Personal Loan')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# loan_negative = train[train['y'] < 0.5]['loan'].value_counts()
# plt.bar(loan_negative.index, loan_negative.values, alpha=0.7, color='red', edgecolor='black')
# plt.title('Loan - y < 0.5')
# plt.xlabel('Personal Loan')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 4. Tasas de conversión por personal loan
# plt.subplot(2, 3, 4)
# conversion_rates = train.groupby('loan')['y'].agg(['mean', 'count'])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Loan')
# plt.xlabel('Personal Loan')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Distribución porcentual
# plt.subplot(2, 3, 5)
# loan_pct = train['loan'].value_counts(normalize=True) * 100
# plt.pie(loan_pct.values, labels=loan_pct.index, autopct='%1.1f%%')
# plt.title('Loan - Distribución %')

# # 6. Comparación lado a lado
# plt.subplot(2, 3, 6)
# comparison_data = pd.DataFrame({
#     'y >= 0.5': train[train['y'] >= 0.5]['loan'].value_counts(),
#     'y < 0.5': train[train['y'] < 0.5]['loan'].value_counts()
# }).fillna(0)
# comparison_data.plot(kind='bar', ax=plt.gca(), color=['green', 'red'], alpha=0.7)
# plt.title('Loan - Comparación')
# plt.xlabel('Personal Loan')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE LOAN ===")
# print("\nTodos los casos:")
# print(train['loan'].value_counts())

# print(f"\nPorcentajes:")
# print((train['loan'].value_counts(normalize=True) * 100).round(1))

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['loan'].value_counts())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['loan'].value_counts())

# print(f"\nTasas de conversión por loan:")
# conversion_stats = train.groupby('loan')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}).sort_values('conversion_rate_%', ascending=False))

In [23]:
# plt.figure(figsize=(15, 8))

# # Crear mapeo de números a días de la semana
# day_mapping = {1: 'Lunes', 2: 'Martes', 3: 'Miércoles', 4: 'Jueves', 5: 'Viernes', 6: 'Sábado', 7: 'Domingo'}
# train['day_name'] = train['day'].map(day_mapping)

# # 1. Distribución general de day
# plt.subplot(2, 3, 1)
# day_counts = train['day_name'].value_counts()
# # Ordenar por día de la semana
# day_order = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
# day_counts = day_counts.reindex([day for day in day_order if day in day_counts.index])
# plt.bar(day_counts.index, day_counts.values, alpha=0.7, edgecolor='black')
# plt.title('Day - Todos los casos')
# plt.xlabel('Día de la Semana')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# day_positive = train[train['y'] >= 0.5]['day_name'].value_counts()
# day_positive = day_positive.reindex([day for day in day_order if day in day_positive.index])
# plt.bar(day_positive.index, day_positive.values, alpha=0.7, color='green', edgecolor='black')
# plt.title('Day - y >= 0.5')
# plt.xlabel('Día de la Semana')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# day_negative = train[train['y'] < 0.5]['day_name'].value_counts()
# day_negative = day_negative.reindex([day for day in day_order if day in day_negative.index])
# plt.bar(day_negative.index, day_negative.values, alpha=0.7, color='red', edgecolor='black')
# plt.title('Day - y < 0.5')
# plt.xlabel('Día de la Semana')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)

# # 4. Tasas de conversión por día de la semana
# plt.subplot(2, 3, 4)
# conversion_rates = train.groupby('day_name')['y'].agg(['mean', 'count'])
# conversion_rates = conversion_rates.reindex([day for day in day_order if day in conversion_rates.index])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Day')
# plt.xlabel('Día de la Semana')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Distribución porcentual
# plt.subplot(2, 3, 5)
# day_pct = train['day_name'].value_counts(normalize=True) * 100
# day_pct = day_pct.reindex([day for day in day_order if day in day_pct.index])
# plt.pie(day_pct.values, labels=day_pct.index, autopct='%1.1f%%')
# plt.title('Day - Distribución %')

# # 6. Comparación lado a lado
# plt.subplot(2, 3, 6)
# comparison_data = pd.DataFrame({
#     'y >= 0.5': train[train['y'] >= 0.5]['day_name'].value_counts(),
#     'y < 0.5': train[train['y'] < 0.5]['day_name'].value_counts()
# }).fillna(0)
# comparison_data = comparison_data.reindex([day for day in day_order if day in comparison_data.index])
# comparison_data.plot(kind='bar', ax=plt.gca(), color=['green', 'red'], alpha=0.7)
# plt.title('Day - Comparación')
# plt.xlabel('Día de la Semana')
# plt.ylabel('Frecuencia')
# plt.xticks(rotation=45)
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE DAY ===")
# print("\nTodos los casos:")
# day_counts_stats = train['day_name'].value_counts()
# day_counts_stats = day_counts_stats.reindex([day for day in day_order if day in day_counts_stats.index])
# print(day_counts_stats)

# print(f"\nPorcentajes:")
# day_pct_stats = (train['day_name'].value_counts(normalize=True) * 100).round(1)
# day_pct_stats = day_pct_stats.reindex([day for day in day_order if day in day_pct_stats.index])
# print(day_pct_stats)

# print(f"\ny >= 0.5:")
# day_pos_stats = train[train['y'] >= 0.5]['day_name'].value_counts()
# day_pos_stats = day_pos_stats.reindex([day for day in day_order if day in day_pos_stats.index])
# print(day_pos_stats)

# print(f"\ny < 0.5:")
# day_neg_stats = train[train['y'] < 0.5]['day_name'].value_counts()
# day_neg_stats = day_neg_stats.reindex([day for day in day_order if day in day_neg_stats.index])
# print(day_neg_stats)

# print(f"\nTasas de conversión por día:")
# conversion_stats = train.groupby('day_name')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# conversion_stats = conversion_stats.reindex([day for day in day_order if day in conversion_stats.index])
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'total_casos': 'total_casos'}).sort_values('conversion_rate_%', ascending=False))

# print(f"\nMapeo utilizado:")
# print("1=Lunes, 2=Martes, 3=Miércoles, 4=Jueves, 5=Viernes, 6=Sábado, 7=Domingo")

In [24]:
# plt.figure(figsize=(15, 8))

# # 1. Distribución general de pdays
# plt.subplot(2, 3, 1)
# plt.hist(train['pdays'], bins=50, alpha=0.7, edgecolor='black')
# plt.title('Pdays - Todos los casos')
# plt.xlabel('Días desde último contacto')
# plt.ylabel('Frecuencia')

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# plt.hist(train[train['y'] >= 0.5]['pdays'], bins=50, alpha=0.7, color='green', edgecolor='black')
# plt.title('Pdays - y >= 0.5')
# plt.xlabel('Días desde último contacto')
# plt.ylabel('Frecuencia')

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# plt.hist(train[train['y'] < 0.5]['pdays'], bins=50, alpha=0.7, color='red', edgecolor='black')
# plt.title('Pdays - y < 0.5')
# plt.xlabel('Días desde último contacto')
# plt.ylabel('Frecuencia')

# # 4. Tasas de conversión por pdays (binned)
# plt.subplot(2, 3, 4)
# # Crear bins especiales para pdays considerando el valor -1 (nunca contactado)
# pdays_bins = [-2, -0.5, 50, 100, 200, 300, 999]
# pdays_labels = ['Never contacted', '0-50', '51-100', '101-200', '201-300', '300+']
# train['pdays_binned'] = pd.cut(train['pdays'], bins=pdays_bins, labels=pdays_labels)
# conversion_rates = train.groupby('pdays_binned')['y'].agg(['mean', 'count'])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Pdays')
# plt.xlabel('Días desde último contacto')
# plt.ylabel('Tasa de Conversión')
# plt.xticks(rotation=45)

# # 5. Boxplot comparativo
# plt.subplot(2, 3, 5)
# # Para el boxplot, filtrar -1 para mejor visualización
# data_to_plot = [
#     train[(train['y'] >= 0.5) & (train['pdays'] != -1)]['pdays'], 
#     train[(train['y'] < 0.5) & (train['pdays'] != -1)]['pdays']
# ]
# box_plot = plt.boxplot(data_to_plot, patch_artist=True, labels=['y >= 0.5', 'y < 0.5'])
# box_plot['boxes'][0].set_facecolor('green')
# box_plot['boxes'][1].set_facecolor('red')
# plt.title('Pdays - Distribución por Clase\n(sin -1)')
# plt.ylabel('Días')

# # 6. Comparación de densidades (sin -1)
# plt.subplot(2, 3, 6)
# train[(train['y'] >= 0.5) & (train['pdays'] != -1)]['pdays'].plot(kind='density', alpha=0.7, color='green', label='y >= 0.5')
# train[(train['y'] < 0.5) & (train['pdays'] != -1)]['pdays'].plot(kind='density', alpha=0.7, color='red', label='y < 0.5')
# plt.title('Pdays - Densidades\n(sin -1)')
# plt.xlabel('Días')
# plt.ylabel('Densidad')
# plt.legend()

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE PDAYS ===")
# print("\nTodos los casos:")
# print(train['pdays'].describe())

# print(f"\nConteo de valor especial -1 (nunca contactado):")
# print(f"Casos con pdays = -1: {(train['pdays'] == -1).sum()}")
# print(f"Porcentaje con pdays = -1: {((train['pdays'] == -1).sum() / len(train) * 100):.1f}%")

# print(f"\nEstadísticas sin -1:")
# pdays_without_minus1 = train[train['pdays'] != -1]['pdays']
# print(pdays_without_minus1.describe())

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['pdays'].describe())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['pdays'].describe())

# print(f"\nTasas de conversión por rangos de pdays:")
# conversion_stats = train.groupby('pdays_binned')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}).sort_values('conversion_rate_%', ascending=False))

# print(f"\nCorrelación con target (incluyendo -1):")
# print(f"Correlación Pearson: {train['pdays'].corr(train['y']):.4f}")

# print(f"\nCorrelación con target (sin -1):")
# correlation_without_minus1 = train[train['pdays'] != -1]['pdays'].corr(train[train['pdays'] != -1]['y'])
# print(f"Correlación Pearson: {correlation_without_minus1:.4f}")

# # Análisis del valor -1
# print(f"\nAnálisis del valor -1 (nunca contactado):")
# never_contacted = train[train['pdays'] == -1]
# previously_contacted = train[train['pdays'] != -1]
# print(f"Tasa conversión nunca contactados: {(never_contacted['y'].mean() * 100):.2f}%")
# print(f"Tasa conversión previamente contactados: {(previously_contacted['y'].mean() * 100):.2f}%")

In [25]:
# plt.figure(figsize=(15, 8))

# # 1. Distribución general de previous
# plt.subplot(2, 3, 1)
# plt.hist(train['previous'], bins=20, alpha=0.7, edgecolor='black')
# plt.title('Previous - Todos los casos')
# plt.xlabel('Número de contactos previos')
# plt.ylabel('Frecuencia')

# # 2. Distribución para y >= 0.5
# plt.subplot(2, 3, 2)
# plt.hist(train[train['y'] >= 0.5]['previous'], bins=20, alpha=0.7, color='green', edgecolor='black')
# plt.title('Previous - y >= 0.5')
# plt.xlabel('Número de contactos previos')
# plt.ylabel('Frecuencia')

# # 3. Distribución para y < 0.5
# plt.subplot(2, 3, 3)
# plt.hist(train[train['y'] < 0.5]['previous'], bins=20, alpha=0.7, color='red', edgecolor='black')
# plt.title('Previous - y < 0.5')
# plt.xlabel('Número de contactos previos')
# plt.ylabel('Frecuencia')

# # 4. Tasas de conversión por número de contactos previos
# plt.subplot(2, 3, 4)
# # Crear categorías para better visualization
# train['previous_cat'] = train['previous'].apply(lambda x: '0' if x == 0 else '1' if x == 1 else '2' if x == 2 else '3+' if x >= 3 else str(x))
# conversion_rates = train.groupby('previous_cat')['y'].agg(['mean', 'count'])
# # Ordenar las categorías
# category_order = ['0', '1', '2', '3+']
# conversion_rates = conversion_rates.reindex([cat for cat in category_order if cat in conversion_rates.index])
# plt.bar(conversion_rates.index, conversion_rates['mean'], alpha=0.7, color='orange', edgecolor='black')
# plt.title('Tasa de Conversión por Previous')
# plt.xlabel('Contactos previos')
# plt.ylabel('Tasa de Conversión')

# # 5. Distribución porcentual de contactos previos
# plt.subplot(2, 3, 5)
# previous_counts = train['previous'].value_counts()
# # Mostrar solo los más frecuentes para mejor visualización
# top_previous = previous_counts.head(8)
# plt.pie(top_previous.values, labels=top_previous.index, autopct='%1.1f%%')
# plt.title('Previous - Top valores %')

# # 6. Boxplot comparativo
# plt.subplot(2, 3, 6)
# data_to_plot = [train[train['y'] >= 0.5]['previous'], train[train['y'] < 0.5]['previous']]
# box_plot = plt.boxplot(data_to_plot, patch_artist=True, labels=['y >= 0.5', 'y < 0.5'])
# box_plot['boxes'][0].set_facecolor('green')
# box_plot['boxes'][1].set_facecolor('red')
# plt.title('Previous - Distribución por Clase')
# plt.ylabel('Contactos previos')

# plt.tight_layout()
# plt.show()

# # ESTADÍSTICAS
# print("=== ESTADÍSTICAS DE PREVIOUS ===")
# print("\nTodos los casos:")
# print(train['previous'].describe())

# print(f"\nDistribución de valores:")
# print(train['previous'].value_counts().sort_index())

# print(f"\nPorcentajes de los valores más frecuentes:")
# previous_pct = (train['previous'].value_counts().head(10) / len(train) * 100).round(1)
# print(previous_pct)

# print(f"\ny >= 0.5:")
# print(train[train['y'] >= 0.5]['previous'].describe())

# print(f"\ny < 0.5:")
# print(train[train['y'] < 0.5]['previous'].describe())

# print(f"\nTasas de conversión por categorías de previous:")
# conversion_stats = train.groupby('previous_cat')['y'].agg(['mean', 'count'])
# conversion_stats['mean'] = (conversion_stats['mean'] * 100).round(2)
# conversion_stats = conversion_stats.reindex([cat for cat in category_order if cat in conversion_stats.index])
# print(conversion_stats.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}))

# print(f"\nTasas de conversión por valores exactos (top 10):")
# conversion_by_value = train.groupby('previous')['y'].agg(['mean', 'count'])
# conversion_by_value['mean'] = (conversion_by_value['mean'] * 100).round(2)
# conversion_by_value = conversion_by_value.sort_values('count', ascending=False).head(10)
# print(conversion_by_value.rename(columns={'mean': 'conversion_rate_%', 'count': 'total_casos'}))

# print(f"\nCorrelación con target:")
# print(f"Correlación Pearson: {train['previous'].corr(train['y']):.4f}")

# print(f"\nAnálisis de casos sin contactos previos (previous = 0):")
# never_contacted = train[train['previous'] == 0]
# previously_contacted = train[train['previous'] > 0]
# print(f"Casos sin contactos previos: {len(never_contacted)} ({(len(never_contacted)/len(train)*100):.1f}%)")
# print(f"Casos con contactos previos: {len(previously_contacted)} ({(len(previously_contacted)/len(train)*100):.1f}%)")
# print(f"Tasa conversión sin contactos previos: {(never_contacted['y'].mean() * 100):.2f}%")
# print(f"Tasa conversión con contactos previos: {(previously_contacted['y'].mean() * 100):.2f}%")

# # Percentiles para identificar outliers
# print(f"\nPercentiles de previous:")
# percentiles = [50, 75, 90, 95, 99, 100]
# for p in percentiles:
#     print(f"P{p}: {train['previous'].quantile(p/100):.0f}")

In [26]:
train.describe(include="all")

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
count,750000.000000,750000.000000,750000,750000,750000,750000,750000.000000,750000,750000,750000,750000.000000,750000,750000.000000,750000.000000,750000.000000,750000.000000,750000,750000.000000
unique,NaN,NaN,12,3,4,2,NaN,2,2,3,NaN,12,NaN,NaN,NaN,NaN,4,NaN
top,NaN,NaN,management,married,secondary,no,NaN,yes,no,cellular,NaN,may,NaN,NaN,NaN,NaN,unknown,NaN
freq,NaN,NaN,175541,480759,401683,737151,NaN,411288,645023,486655,NaN,228411,NaN,NaN,NaN,NaN,672450,NaN
mean,374999.500000,40.926395,NaN,NaN,NaN,NaN,1204.067397,NaN,NaN,NaN,16.117209,NaN,256.229144,2.577008,22.412733,0.298545,NaN,0.120651
std,216506.495284,10.098829,NaN,NaN,NaN,NaN,2836.096759,NaN,NaN,NaN,8.250832,NaN,272.555662,2.718514,77.319998,1.335926,NaN,0.325721
min,0.000000,18.000000,NaN,NaN,NaN,NaN,-8019.000000,NaN,NaN,NaN,1.000000,NaN,1.000000,1.000000,-1.000000,0.000000,NaN,0.000000
25%,187499.750000,33.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,9.000000,NaN,91.000000,1.000000,-1.000000,0.000000,NaN,0.000000
50%,374999.500000,39.000000,NaN,NaN,NaN,NaN,634.000000,NaN,NaN,NaN,17.000000,NaN,133.000000,2.000000,-1.000000,0.000000,NaN,0.000000
75%,562499.250000,48.000000,NaN,NaN,NaN,NaN,1390.000000,NaN,NaN,NaN,21.000000,NaN,361.000000,3.000000,-1.000000,0.000000,NaN,0.000000


In [27]:
# agregar
# columna month_succes_rate valores entre 0 y 1
#       total_contacts  conversions  conversion_rate
#month                                              
#mar              5802         3315         0.571355
#sep              7409         3962         0.534755
#dec              2069         1062         0.513291
#oct              9204         4510         0.490004
#apr             41319         9737         0.235654
#feb             37611         7778         0.206801
#jan             18937         2351         0.124148
#aug            128859        14453         0.112161
#nov             66062         7254         0.109806
#jun             93670         9716         0.103726
#jul            110647        10052         0.090847
#may            228411        16298         0.071354
# balance inferior a 565 mas duration inferior a 203.8(media de los que no) y contact unknown ,crear columna binaria que se llame Non-interested
#Tasas de conversión por rangos de age: crear columna S-R-U , personas menos de 25 o mayores de 65 , y que job sea student or retired or unemployed
# columna binaria con 1 y 0 
#       conversion_rate_%  total_casos
#age                                  
#<25                26.92        16860
#25-34              13.04       260150
#35-44               9.64       233150
#45-54               9.14       164337
#55-64              15.26        67370
#65+                52.19         8133
#Tasas de conversión por job:
#               conversion_rate_%  total_casos
#job                                          
#student                    34.08        11767
#retired                    24.62        35185
#unemployed                 17.98        17634
#management                 15.04       175541
#self-employed              12.94        19020
#unknown                    12.07         2917
#technician                 11.83       138107
#admin.                     11.65        81492
#housemaid                   8.47        15912
#services                    8.27        64209
#entrepreneur                8.14        17718
#blue-collar                 6.74       170498
#Tasas de conversión por marital:
#          conversion_rate_%  total_casos
#marital                                 
#single                17.05       194834
#divorced              11.16        74407
#married               10.19       480759
#Tasas de conversión por education:
#           conversion_rate_%  total_casos
#education                                
#tertiary               16.26       227508
#unknown                13.34        21299
#secondary              10.55       401683
#primary                 8.31        99510
# crear columna con education tertiary + marital = single ,binaria llamarla TM
#Tasas de conversión por housing:
#         conversion_rate_%  total_casos
#housing                                
#no                   17.58       338712
#yes                   7.53       411288
# crear columna binaria de housing yes o default yes o loan yes llamarla Financia_issues
#Tasas de conversión por día:
#           conversion_rate_%  count
#day_name                           
#Lunes                  35.81   3890
#Miércoles              20.42  15827
#Jueves                 18.60  22270
#Martes                 17.53  20003
#Viernes                13.27  30245
#Domingo                 9.82  28771
#Sábado                  9.35  30573
#crear columna binaria para fin de semana ,llamaral Weekend
#Tasas de conversión por rangos de pdays:
#                 conversion_rate_%  total_casos
#pdays_binned                                   
#51-100                       64.73        12454
#101-200                      33.92        29690
#0-50                         19.88          845
#300+                         10.73        24536
#201-300                      10.29        10041
#Never contacted              10.19       672434
#Tasa conversión nunca contactados: 10.19%
#Tasa conversión previamente contactados: 28.32%
# crear columna para pdays = -1 o pdays>200  o previous = 0, binaria 0/1 llamarla Insistencia
#previous                                
#0                         10.19       672431
#1                         25.74        28342
#2                         26.21        20468
#3+                        32.37        28759



In [28]:
#pipeline One-hot
#para transformar train y test , para poder trabajar

# class BankMarketingPreprocessor:
#     def __init__(self):
#         self.job_categories = None
#         self.fitted = False
    
#     def fit_transform(self, train_df):
#         """
#         #Ajusta el preprocessor en los datos de entrenamiento y los transforma
#         """
#         df = train_df.copy()
        
#         # 1. TARGET VARIABLE (ya viene como 0/1, no modificar)
#         # df['y'] ya está en formato correcto
        
#         # 2. VARIABLES BINARIAS (Label Encoding simple)
#         binary_mappings = {
#             'default': {'no': 0, 'yes': 1},
#             'housing': {'no': 0, 'yes': 1}, 
#             'loan': {'no': 0, 'yes': 1}
#         }
        
#         for col, mapping in binary_mappings.items():
#             df[col] = df[col].map(mapping)
        
#         # 3. EDUCATION (Ordinal)
#         education_mapping = {
#             'unknown': 0,
#             'primary': 1, 
#             'secondary': 2, 
#             'tertiary': 3
#         }
#         df['education'] = df['education'].map(education_mapping)
        
#         # 4. ONE-HOT ENCODING
#         # Contact (3 categorías -> 2 columnas)
#         df['contact_cellular'] = (df['contact'] == 'cellular').astype(int)
#         df['contact_telephone'] = (df['contact'] == 'telephone').astype(int)
#         # unknown se infiere cuando ambas son 0
        
#         # Month (12 categorías -> 11 columnas)
#         months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 
#                  'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        
#         for month in months[1:]:  # Omitimos 'jan' como referencia
#             df[f'month_{month}'] = (df['month'] == month).astype(int)
        
#         # Poutcome (4 categorías -> 3 columnas)
#         poutcomes = ['failure', 'other', 'success']  # Omitimos 'unknown' como referencia
#         for outcome in poutcomes:
#             df[f'poutcome_{outcome}'] = (df['poutcome'] == outcome).astype(int)
        
#         # Job (12 categorías -> 11 columnas) - GUARDAR CATEGORÍAS PARA TEST
#         self.job_categories = sorted(df['job'].unique())  # Ordenar para consistencia
#         reference_job = self.job_categories[0]  # Primera categoría como referencia
        
#         for job in self.job_categories:
#             if job != reference_job:
#                 df[f'job_{job}'] = (df['job'] == job).astype(int)
        
#         # Marital (3 categorías -> 2 columnas) 
#         maritals = ['single', 'divorced']  # Omitimos 'married' como referencia
#         for marital in maritals:
#             df[f'marital_{marital}'] = (df['marital'] == marital).astype(int)
        
#         # 5. MANTENER pdays ORIGINAL
#         # No hacer nada con pdays - mantenerlo como está (-1 para no contactados)
        
#         # 6. ELIMINAR COLUMNAS ORIGINALES CATEGÓRICAS
#         columns_to_drop = ['job', 'marital', 'contact', 'month', 'poutcome']
#         df = df.drop(columns=columns_to_drop)
        
#         self.fitted = True
#         return df
    
#     def transform(self, test_df):
#         """
#         #Transforma los datos de test usando los parámetros ajustados en train
#         """
#         if not self.fitted:
#             raise ValueError("El preprocessor debe ser ajustado primero con fit_transform()")
        
#         df = test_df.copy()
        
#         # 1. VARIABLES BINARIAS (mismas que en train)
#         binary_mappings = {
#             'default': {'no': 0, 'yes': 1},
#             'housing': {'no': 0, 'yes': 1}, 
#             'loan': {'no': 0, 'yes': 1}
#         }
        
#         for col, mapping in binary_mappings.items():
#             df[col] = df[col].map(mapping)
        
#         # 2. EDUCATION (Ordinal - mismo que en train)
#         education_mapping = {
#             'unknown': 0,
#             'primary': 1, 
#             'secondary': 2, 
#             'tertiary': 3
#         }
#         df['education'] = df['education'].map(education_mapping)
        
#         # 3. ONE-HOT ENCODING (mismo que en train)
#         # Contact
#         df['contact_cellular'] = (df['contact'] == 'cellular').astype(int)
#         df['contact_telephone'] = (df['contact'] == 'telephone').astype(int)
        
#         # Month (mismos meses que en train)
#         months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 
#                  'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        
#         for month in months[1:]:  # Omitimos 'jan' como referencia
#             df[f'month_{month}'] = (df['month'] == month).astype(int)
        
#         # Poutcome (mismas categorías que en train)
#         poutcomes = ['failure', 'other', 'success']  # Omitimos 'unknown' como referencia
#         for outcome in poutcomes:
#             df[f'poutcome_{outcome}'] = (df['poutcome'] == outcome).astype(int)
        
#         # Job (USAR EXACTAMENTE LAS MISMAS CATEGORÍAS QUE EN TRAIN)
#         if self.job_categories is None:
#             raise ValueError("Las categorías de job no fueron guardadas durante fit_transform()")
        
#         reference_job = self.job_categories[0]  # Misma referencia que en train
        
#         for job in self.job_categories:
#             if job != reference_job:
#                 df[f'job_{job}'] = (df['job'] == job).astype(int)
        
#         # Marital (mismas categorías que en train)
#         maritals = ['single', 'divorced']  # Omitimos 'married' como referencia
#         for marital in maritals:
#             df[f'marital_{marital}'] = (df['marital'] == marital).astype(int)
        
#         # 4. MANTENER pdays ORIGINAL
#         # No hacer nada con pdays - mantenerlo como está
        
#         # 5. ELIMINAR COLUMNAS ORIGINALES CATEGÓRICAS (mismas que en train)
#         columns_to_drop = ['job', 'marital', 'contact', 'month', 'poutcome']
#         df = df.drop(columns=columns_to_drop)
        
#         return df



In [29]:
#pipeline One-hot V2

class BankMarketingPreprocessor:
    def __init__(self):
        self.job_categories = None
        self.fitted = False

    def _apply_common_transformations(self, df_original):
        """
        MÉTODO PRIVADO: Contiene TODAS las transformaciones comunes
        para evitar duplicación y errores.
        """
        df = df_original.copy()

        # --- Feature Engineering ---
        month_success_rates = {
            'mar': 0.571355, 'sep': 0.534755, 'dec': 0.513291, 'oct': 0.490004,
            'apr': 0.235654, 'feb': 0.206801, 'jan': 0.124148, 'aug': 0.112161,
            'nov': 0.109806, 'jun': 0.103726, 'jul': 0.090847, 'may': 0.071354
        }
        df['month_success_rate'] = df['month'].map(month_success_rates)
        df['Non_interested'] = ((df['balance'] < 565) & (df['duration'] < 203.8) & (df['contact'] == 'unknown')).astype(int)
        df['S_R_U'] = (((df['age'] < 25) | (df['age'] >= 65)) & (df['job'].isin(['student', 'retired', 'unemployed']))).astype(int)
        df['TM'] = ((df['education'] == 'tertiary') & (df['marital'] == 'single')).astype(int)
        df['Financial_issues'] = ((df['housing'] == 'yes') | (df['default'] == 'yes') | (df['loan'] == 'yes')).astype(int)
        
        day_mapping = {1: 'Lunes', 2: 'Martes', 3: 'Miércoles', 4: 'Jueves', 5: 'Viernes', 6: 'Sábado', 7: 'Domingo'}
        df['day_name'] = df['day'].map(day_mapping) # Columna auxiliar
        df['Weekend'] = (df['day_name'].isin(['Viernes','Sábado', 'Domingo'])).astype(int)
        
        df['Insistencia'] = ((df['pdays'] == -1) | (df['pdays'] > 200) | (df['previous'] == 0)).astype(int)

        # --- Codificación de Variables ---
        binary_mappings = {'default': {'no': 0, 'yes': 1}, 'housing': {'no': 0, 'yes': 1}, 'loan': {'no': 0, 'yes': 1}}
        for col, mapping in binary_mappings.items():
            df[col] = df[col].map(mapping)

        education_mapping = {'unknown': 0, 'primary': 1, 'secondary': 2, 'tertiary': 3}
        df['education'] = df['education'].map(education_mapping)

        # --- One-Hot Encoding ---
        df['contact_cellular'] = (df['contact'] == 'cellular').astype(int)
        df['contact_telephone'] = (df['contact'] == 'telephone').astype(int)
        
        months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        for month in months[1:]:
            df[f'month_{month}'] = (df['month'] == month).astype(int)
            
        poutcomes = ['failure', 'other', 'success']
        for outcome in poutcomes:
            df[f'poutcome_{outcome}'] = (df['poutcome'] == outcome).astype(int)
        
        maritals = ['single', 'divorced']
        for marital in maritals:
            df[f'marital_{marital}'] = (df['marital'] == marital).astype(int)
        
        # OHE para 'job' usando las categorías guardadas del set de train
        reference_job = self.job_categories[0]
        for job in self.job_categories:
            if job != reference_job:
                df[f'job_{job}'] = (df['job'] == job).astype(int)
        
        # --- Limpieza Final ---
        # Se eliminan las columnas originales y la auxiliar 'day_name'
        columns_to_drop = ['job', 'marital', 'contact', 'month', 'poutcome', 'day_name']
        df = df.drop(columns=columns_to_drop)
        
        return df

    def fit_transform(self, train_df):
        """Ajusta y transforma los datos de entrenamiento."""
        # APRENDE las categorías de 'job' del set de entrenamiento
        self.job_categories = sorted(train_df['job'].unique())
        
        # APLICA todas las transformaciones llamando al método común
        df_transformed = self._apply_common_transformations(train_df)
        
        self.fitted = True
        return df_transformed
        
    def transform(self, test_df):
        """Transforma los datos de test."""
        if not self.fitted:
            raise ValueError("El preprocessor debe ser ajustado primero con fit_transform()")
        
        # APLICA EXACTAMENTE las mismas transformaciones que en train
        df_transformed = self._apply_common_transformations(test_df)
        
        return df_transformed
# NUEVAS FEATURES AGREGADAS:
# - month_success_rate: Tasa de éxito histórica por mes (0-1)
# - Non_interested: Clientes con bajo balance, duración corta y contacto desconocido
# - S_R_U: Estudiantes, jubilados o desempleados jóvenes (<25) o mayores (>=65)
# - TM: Educación terciaria y solteros
# - Financial_issues: Tienen algún tipo de préstamo o problema crediticio
# - Weekend: Contactados en fin de semana
# - Insistencia: Casos de baja insistencia (nunca contactados, hace mucho tiempo, o sin contactos previos)

In [30]:
preprocessor = BankMarketingPreprocessor()
              
X_train_p = preprocessor.fit_transform(train)
X_test_p = preprocessor.transform(test)

y_train_f = X_train_p["y"]
X_train_f = X_train_p.drop("y", axis=1)

print(f"Shape final train: {X_train_f.shape}")
print(f"Shape final test: {X_test_p.shape}")
print(f"Columnas finales: {list(X_train_f.columns)}")
print(f"Columnas finales: {list(X_test_p.columns)}")

Shape final train: (750000, 48)
Shape final test: (250000, 48)
Columnas finales: ['id', 'age', 'education', 'default', 'balance', 'housing', 'loan', 'day', 'duration', 'campaign', 'pdays', 'previous', 'month_success_rate', 'Non_interested', 'S_R_U', 'TM', 'Financial_issues', 'Weekend', 'Insistencia', 'contact_cellular', 'contact_telephone', 'month_feb', 'month_mar', 'month_apr', 'month_may', 'month_jun', 'month_jul', 'month_aug', 'month_sep', 'month_oct', 'month_nov', 'month_dec', 'poutcome_failure', 'poutcome_other', 'poutcome_success', 'marital_single', 'marital_divorced', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown']
Columnas finales: ['id', 'age', 'education', 'default', 'balance', 'housing', 'loan', 'day', 'duration', 'campaign', 'pdays', 'previous', 'month_success_rate', 'Non_interested', 'S_R_U', 'TM', 'Financial_issues', 'Weekend', 'I

In [31]:
X_train_p.head()

,id,age,education,default,balance,housing,loan,day,duration,campaign,pdays,previous,y,month_success_rate,Non_interested,S_R_U,TM,Financial_issues,Weekend,Insistencia,contact_cellular,contact_telephone,month_feb,month_mar,month_apr,month_may,month_jun,month_jul,month_aug,month_sep,month_oct,month_nov,month_dec,poutcome_failure,poutcome_other,poutcome_success,marital_single,marital_divorced,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown
0,0,42,2,0,7,0,0,25,117,3,-1,0,0,0.112161,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,38,2,0,514,0,0,18,185,1,-1,0,0,0.103726,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,36,2,0,602,1,0,14,111,2,-1,0,0,0.071354,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,27,2,0,34,1,0,28,10,2,-1,0,0,0.071354,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,4,26,2,0,889,1,0,3,902,1,-1,0,1,0.206801,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [32]:
X_test_p.head()

,id,age,education,default,balance,housing,loan,day,duration,campaign,pdays,previous,month_success_rate,Non_interested,S_R_U,TM,Financial_issues,Weekend,Insistencia,contact_cellular,contact_telephone,month_feb,month_mar,month_apr,month_may,month_jun,month_jul,month_aug,month_sep,month_oct,month_nov,month_dec,poutcome_failure,poutcome_other,poutcome_success,marital_single,marital_divorced,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown
0,750000,32,2,0,1397,1,0,21,224,1,-1,0,0.071354,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,750001,44,3,0,23,1,0,3,586,2,-1,0,0.235654,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,750002,36,1,0,46,1,1,13,111,2,-1,0,0.071354,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,750003,58,2,0,-1380,1,1,29,125,1,-1,0,0.071354,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,750004,28,2,0,1950,1,0,22,181,1,-1,0,0.090847,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [33]:
# Guardar IDs antes de eliminar
train_ids = X_train_f['id'].copy()
test_ids = X_test_p['id'].copy()

# Eliminar ID de los datasets
X_train_final = X_train_f.drop('id', axis=1)  # 40 columnas
X_test_final = X_test_p.drop('id', axis=1)    # 40 columnas

print(f"Train shape sin ID: {X_train_final.shape}")
print(f"Test shape sin ID: {X_test_final.shape}")

Train shape sin ID: (750000, 47)
Test shape sin ID: (250000, 47)


In [47]:
%%time
xgb_model7 = xgb.XGBClassifier(
        n_estimators=475,
        learning_rate=0.0718,
        max_depth=9,
        subsample=0.8006,
        colsample_bytree=0.8169,
        colsample_bylevel=0.7323,
        min_child_weight=7,
        gamma=0.4794,
        reg_alpha=0.6923,
        reg_lambda=0.9114,
        random_state=42,
        n_jobs=-1,
        eval_metric='auc',
        use_label_encoder=False,
        verbosity=2
)    
# Entrenar
xgb_model7.fit(X_train_final, y_train_f)

# Hacer predicciones
y_test_pred_xgb7 = xgb_model7.predict_proba(X_test_final)[:, 1]

# #Submission
# submission = pd.DataFrame({
#    'id': test_ids,
#    'y': y_test_pred_xgb7
# })
# submission.to_csv('submission.csv', index=False)

# submission.head()

CPU times: user 2min 27s, sys: 396 ms, total: 2min 28s
Wall time: 38.6 s


,id,y
0,750000,0.002139
1,750001,0.078298
2,750002,0.000150
3,750003,0.000074
4,750004,0.009429


In [48]:
%%time
from lightgbm import LGBMClassifier

lgb_model5 = LGBMClassifier(
    random_state=42,
    n_jobs=-5,
    objective='binary',  
    boosting_type='gbdt',
    metric='binary_logloss',
    n_estimators=366,
    max_depth=-1,
    learning_rate=0.0702,
    num_leaves=148,
    subsample=0.9878,
    colsample_bytree=0.7205,
    min_child_samples=36,
    reg_alpha=0.3872,
    reg_lambda=0.9090
)

# Entrenar
lgb_model5.fit(X_train_final, y_train_f)

# Hacer predicciones
y_test_pred_lgb5 = lgb_model5.predict_proba(X_test_final)[:, 1]

# #Submission
# submission = pd.DataFrame({
#    'id': test_ids,
#    'y': y_test_pred
# })
# submission.to_csv('submission.csv', index=False)

# submission.head()

[LightGBM] [Info] Number of positive: 90488, number of negative: 659512
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.145610 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1050
[LightGBM] [Info] Number of data points in the train set: 750000, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120651 -> initscore=-1.986283
[LightGBM] [Info] Start training from score -1.986283
CPU times: user 55.7 s, sys: 452 ms, total: 56.2 s
Wall time: 56.7 s


In [49]:
# Ensemble simple promediando probabilidades
y_test_pred_ensemble = (y_test_pred_lgb5 + y_test_pred_xgb7) / 2

# Crear el submission
submission = pd.DataFrame({
    'id': test_ids,
    'y': y_test_pred_ensemble
})
submission.to_csv('submission.csv', index=False)

submission.head()


,id,y
0,750000,0.002452
1,750001,0.096378
2,750002,0.000204
3,750003,0.000068
4,750004,0.010330


In [50]:
submission.describe()

,id,y
count,250000.000000,250000.000000
mean,874999.500000,0.120287
std,72168.927986,0.246485
min,750000.000000,0.000008
25%,812499.750000,0.000356
50%,874999.500000,0.001863
75%,937499.250000,0.075007
max,999999.000000,0.998180


In [ ]:
#v1 150 iteraciones  CV Score: 0.9613 en competencia 0.89948 ,el primero tiene 0.97778 7/8/2025
#rf1 = RandomForestClassifier( #rf target encod 150 iter             
#    n_estimators=100,                   #
#    min_samples_split=500,              #
#    min_samples_leaf=200,               #
#    max_features=0.3,                   #
#    max_depth= None,                    #
#    criterion = "entropy",              #
#    bootstrap = False,                  #
#    random_state=42,                    #
#    n_jobs=-1,                          #
#    class_weight='balanced'             #
#    )
"""=== FEATURE IMPORTANCE (TOP 10) ===
             feature  importance
7           duration    0.647334
3            balance    0.082880
26  poutcome_success    0.040380
11  contact_cellular    0.038691
4            housing    0.037123
9              pdays    0.020095
0                age    0.016717
6                day    0.014675
14         month_mar    0.013597
10          previous    0.009041
"""
#v2 XGB 250 iteraciones CV Score: 0.9676 en competencia 0.96850
#xgb_model1 = xgb.XGBClassifier(
#        n_estimators=366,              #
#        learning_rate=0.1316,          #
#        max_depth=8,                   #
#        subsample=0.8724,              #
#        colsample_bytree=0.7738,       #
#        colsample_bylevel=0.7321,      #
#        min_child_weight=4,            #
#        gamma=0.1268,                  #
#        reg_alpha=0.8228,              #
#        reg_lambda=0.7442,             #
#        random_state=42,               #
#        n_jobs=-1,                     #
#        eval_metric='auc',             #
#        use_label_encoder=False,       #
#        verbosity=0                    #
#) 
"""
target Encoding v1 rfe
=== MEJORES PARÁMETROS ===
{'n_estimators': 200, 'min_samples_split': 100, 'min_samples_leaf': 50, 'max_features': 0.3, 'max_depth': 20, 'criterion': 'gini',
 'bootstrap': False}

=== MEJOR SCORE (AUC) ===
CV Score: 0.9620

=== FEATURE IMPORTANCE (TOP 10) ===
                 feature  importance
7               duration    0.651461
3                balance    0.076133
18  month_target_encoded    0.071657
15      poutcome_success    0.038616
11      contact_cellular    0.034448
4                housing    0.033808
6                    day    0.017222
9                  pdays    0.015846
0                    age    0.013047
16    job_target_encoded    0.011721
"""

#v3 xgb 2500 iteraciones One hot y xgb 2500 iteracion Target Encod , CV Score: 0.9676 en One-hot y target, en competencia 0.96888
# si o si ver de feature eng y/o una NN para continuar
# Probe NN , varias estructuras y no es viable para nada ,en competencia lograba 0.82 promedio , muy mal
#xgb_model2 = xgb.XGBClassifier(
#        n_estimators=475,          # 424
#        learning_rate=0.0718,      # 0.0896
#        max_depth=9,               # 9
#        subsample=0.8006,          # 0.9350
#        colsample_bytree=0.8169,   # 0.7346
#        colsample_bylevel=0.7323,  # 0.7502
#        min_child_weight=7,        # 4
#        gamma=0.4794,              # 0.2506
#        reg_alpha=0.6923,          # 0.6156
#        reg_lambda=0.9114,         # 0.8680
#        random_state=42,           # 42
#        n_jobs=-1,                 #
#        eval_metric='auc',         #
#        use_label_encoder=False,   #
#        verbosity=0                #
#)
#Mejor score (CV): 0.9678
#   Feature Importance XGB
# 26   poutcome_success    0.295378
# 14          month_mar    0.095745
# 7            duration    0.074167
# 11   contact_cellular    0.067707
# 4             housing    0.044851
# 21          month_oct    0.037620
# 15          month_apr    0.034245
# 17          month_jun    0.033480
# 20          month_sep    0.030168
# 12  contact_telephone    0.024734
# 5                loan    0.022850
# 27    job_blue-collar    0.020831
# 16          month_may    0.019547
# 13          month_feb    0.017789
# 18          month_jul    0.014844
# 22          month_nov    0.013827
# 23          month_dec    0.013768
# 19          month_aug    0.012835
# 9               pdays    0.009650
# 10           previous    0.009134



#v4 con feat eng , xgb 25 iteraciones one-hot (CV): 0.9673 en competencia 0.9685 en competencia , por ahora mejor el anterior 
#falta que pruebe con mas de 1000 iteraciones para ver si mejora mas
# xgb_model4 = xgb.XGBClassifier(
#         n_estimators=377,
#         learning_rate=0.0447,
#         max_depth=9,
#         subsample=0.7483,
#         colsample_bytree=0.6296,
#         colsample_bylevel=0.9085,
#         min_child_weight=9,
#         gamma=0.1792,
#         reg_alpha=0.4494,
#         reg_lambda=0.0954,
#         random_state=42,
#         n_jobs=-1,
#         eval_metric='auc',
#         use_label_encoder=False,
#         verbosity=0
# )
#               feature  importance
# 33    poutcome_success    0.226028
# 12      Non_interested    0.155270
# 7             duration    0.079444
# 17         Insistencia    0.051111
# 18    contact_cellular    0.047980
# 4              housing    0.042457
# 15    Financial_issues    0.040018
# 24           month_jun    0.029941
# 11  month_success_rate    0.024906
# 21           month_mar    0.024538
# 13               S_R_U    0.019711
# 19   contact_telephone    0.017600
# 26           month_aug    0.015405
# 22           month_apr    0.015267
# 28           month_oct    0.014416
# 5                 loan    0.013715
# 29           month_nov    0.012797
# 36     job_blue-collar    0.011803
# 20           month_feb    0.010032
# 27           month_sep    0.008990

# viendo si el Feat Eng tuvo efecto

#v5 lgb 1750 iteraciones Mejor score (CV): 0.9685 en competencia 0.96972
# lgb_model5 = LGBMClassifier(
#     random_state=42,
#     n_jobs=-5,
#     objective='binary',  
#     boosting_type='gbdt',
#     metric='binary_logloss',
#     n_estimators=366,
#     max_depth=-1,
#     learning_rate=0.0702,
#     num_leaves=148,
#     subsample=0.9878,
#     colsample_bytree=0.7205,
#     min_child_samples=36,
#     reg_alpha=0.3872,
#     reg_lambda=0.9090
# )

#v6 no cambie parametros

#v7 xgb 1750 iter Mejor score (CV): 0.9679  , en competencia 0.96914 en competencia

# xgb_model6 = xgb.XGBClassifier(
#         n_estimators=475,
#         learning_rate=0.0718,
#         max_depth=9,
#         subsample=0.8006,
#         colsample_bytree=0.8169,
#         colsample_bylevel=0.7323,
#         min_child_weight=7,
#         gamma=0.4794,
#         reg_alpha=0.6923,
#         reg_lambda=0.9114,
#         random_state=42,
#         n_jobs=-1,
#         eval_metric='auc',
#         use_label_encoder=False,
#         verbosity=2

#v8 ensemble lgb 1750 iter Mejor score (CV): 0.9685 + xgb 1750 iter Mejor score (CV): 0.9679  , en competencia 0.96985
